In [43]:
import pandas as pd
import numpy as np
import scipy.sparse as ssp

from DSI_Capstone_Steemit.utils.utils import(
    load_data_and_description,
)
from sklearn.model_selection import train_test_split

import os

data_directory = '../data/'
input_directory = os.path.join(data_directory,'networkx_votes')

def load_joblib(filename):
    return joblib.load(os.path.join(input_directory,filename))

data,feature_names,data_desc = load_data_and_description(data_type='tfidf')
data_desc['log total_payout_value'] = np.log(data_desc['total_payout_value'])



Loading TFIDF Tags


In [44]:
idx = data_desc['depth'] == 0
data_desc = data_desc[idx]
data = data[idx.values]
data_desc['created'] = pd.DatetimeIndex(data_desc['created'])
idx = (data_desc['created'] > pd.to_datetime('2016-11-01'))
data = data[idx.values]
data_desc = data_desc[idx]




In [45]:
import joblib

In [46]:
print data.shape,data_desc.shape

(31485, 21264) (31485, 72)


In [70]:
network_cols = ['Core K', u'Eig Cent',
       u'Hubs', u'Authorities']
# Remove middle value articles

# idx1 = data_desc['log total_payout_value'] < 1.2
# idx2 = data_desc['log total_payout_value'] >2.5

# idx_not = (~idx1) & (~idx2)

# data_desc = data_desc[~idx_not]
# data = data[~idx_not.values,:]
y = data_desc['log total_payout_value']

# For Regression
# y = data_desc['log total_payout_value']
value_counts = data_desc['category'].value_counts()
top_categories = value_counts.index[value_counts > np.percentile(data_desc['category'].value_counts(),97)]
idx = data_desc['category'].isin(top_categories)
data_desc['top category'] = idx.astype(int)

data_desc['top category listed'] = data_desc.ix[data_desc['top category'].values.astype(bool) ,'category']

data_desc['top category listed'] = data_desc['top category listed'].fillna('Other')


post_features = ['number of body tags',
                                   'number of body urls',
                                   'number of image urls',
                                   'number of body mentions',
                                   'number of image urls',
                                   'number of youtube urls',
                                   'language',
                                   'author_reputation_scaled',
                                   'number of steem counts',
                                'top category','Gradient'] + network_cols


train_features = data_desc.ix[:,post_features].fillna(0)

train = pd.get_dummies(train_features,columns=['language'])

num_image_urls = train['number of image urls'].values[:,0]
train.drop('number of image urls',axis = 1, inplace=True)

train['number of image urls'] = num_image_urls

training_names = train.columns

train_sparse = ssp.csr_matrix(train)
new_data = ssp.hstack([data,train_sparse])
train = new_data.tocsr()

# All samples
number_of_samples = train.shape[0]

X_train, X_test, y_train, y_test = train_test_split(
    train, y, test_size=0.33, random_state=42)


In [71]:
X_train.shape

(21094, 21309)

In [72]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy


In [73]:
# samples = X_train
X = X_train.toarray()#[0:samples]
y = y_train.values#[0:samples]
# y = keras.utils.np_utils.to_categorical(y)



X_val = X_test.toarray()#[0:samples]

y_val = y_test.values#[0:samples]
# y_val = keras.utils.np_utils.to_categorical(y_val)




In [74]:
# from sklearn.feature_selection import f_regression
# f_values = f_regression(X,y_train.values)
# values,pvalues = f_values
# idx = (pvalues < 0.005)
# X_val = X_val[:,idx]
# X = X[:,idx]

In [75]:
all_features = np.array(feature_names + list(training_names))

In [57]:
print X.shape
print X_val.shape

(21094, 233)
(10391, 233)


In [58]:
X

array([[  0.,   0.,   0., ...,   0.,   0.,  27.],
       [  0.,   0.,   0., ...,   0.,   0.,   6.],
       [  0.,   0.,   0., ...,   0.,   0.,   4.],
       ..., 
       [  0.,   0.,   0., ...,   0.,   0.,  48.],
       [  0.,   0.,   0., ...,   0.,   0.,   1.],
       [  0.,   0.,   0., ...,   0.,   0.,   9.]])

In [69]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler(copy = False)

In [77]:
X_train = ss.fit_transform(X_train.toarray())

In [78]:
X_test = ss.transform(X_test.toarray())

## Feature Selection

In [79]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [80]:
model = Sequential()
model.add(Dense(100, input_dim=X.shape[1], activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))


model.add(Dense(1))
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam',validation_data = (X_val,y_val),lr=0.0001)



In [93]:
model.fit(X,y)



Epoch 1/10
21094/21094 [==============================] - 24s - loss: 0.9089    
Epoch 2/10
21094/21094 [==============================] - 24s - loss: 0.8875    
Epoch 3/10
21094/21094 [==============================] - 24s - loss: 0.8684    
Epoch 4/10
21094/21094 [==============================] - 24s - loss: 0.8793    
Epoch 5/10
21094/21094 [==============================] - 24s - loss: 0.8519    
Epoch 6/10
21094/21094 [==============================] - 24s - loss: 0.8592    
Epoch 7/10
21094/21094 [==============================] - 24s - loss: 0.8452    
Epoch 8/10
21094/21094 [==============================] - 24s - loss: 0.8357    
Epoch 9/10
21094/21094 [==============================] - 24s - loss: 0.8378    
Epoch 10/10
21094/21094 [==============================] - 24s - loss: 0.8147    


In [94]:
y_val_pred = model.predict(X_val)

In [95]:
from sklearn.metrics import r2_score

In [96]:
r2_score(y_pred=y_val_pred,y_true=y_val)

0.13175692784775528

In [97]:
#0.23192681053521336 100,100